In [2]:
import requests 
import json 
import pandas as pd
import time
import urllib
import urllib.request 
from urllib.error import HTTPError
from bs4 import *
import os
import unidecode

save_dir=r"C:\Users\mihnea.andrei\Python scripts\resnet_andrei\wiki_face_parser\images"

def timming(since):
    now=time.time() 
    s=now-since
    m=int(s/60)
    s=s-m*60
    return "%dm %ds"%(m,s)

In [3]:
count_links=694700
count_people=0
start=time.time()
printing_iter=10**(2)
file1=open(r"C:\Users\mihnea.andrei\Python scripts\resnet_andrei\wiki_face_parser\names\Print_details.txt","w")
print_data=""
image_name=""

while True:
    prefix="Q"+str(count_links)
    flag=1
    wiki_link="https://www.wikidata.org/wiki/%s"%(prefix)
    json_link="https://www.wikidata.org/wiki/Special:EntityData/%s.json"%(prefix) 
    try:
        response=requests.get(json_link)
    except requests.exceptions.Timeout:
        print("Too much time to request wikipedia link %s"%(image_link))
        flag=0
    except requests.exceptions.TooManyRedirects:
        print("Bad wikipedia link %s"%(image_link))
        flag=0
    except requests.exceptions.RequestException as e:
        print("Catastrophic error for wikipaedia link %s"%(image_link))
        flag=0
    if flag==1:
        try: 
            person=json.loads(response.content)
        except ValueError as err:
            flag=0
    if flag==1:
        if prefix in person["entities"].keys(): 
            if "P31" in person["entities"][prefix]["claims"].keys():
                if "mainsnak" in person["entities"][prefix]["claims"]["P31"][0].keys():
                    if "datavalue" in person["entities"][prefix]["claims"]["P31"][0]["mainsnak"].keys():
                        if "value" in person["entities"][prefix]["claims"]["P31"][0]["mainsnak"]["datavalue"].keys():
                            if "id" in person["entities"][prefix]["claims"]["P31"][0]["mainsnak"]["datavalue"]["value"].keys():
                                if person["entities"][prefix]["claims"]["P31"][0]["mainsnak"]["datavalue"]["value"]["id"]=="Q5":
                                    #############3
                                    #Obtain the name
                                    ##############3
                                    person_name=""
                                    if "entities" in person.keys():   
                                        if prefix in person["entities"].keys():
                                            if "labels" in person["entities"][prefix].keys(): 
                                                if "en" in person["entities"][prefix]["labels"].keys():
                                                    person_name=person["entities"][prefix]["labels"]["en"]["value"]
                                                    #remove quotes since it gives an error when the image file name when saved
                                                    if '"' in person_name:
                                                        person_name=person_name.replace('"',"")
                                                    if "/" in person_name:
                                                        person_name=person_name.replace('/',"")
                                                    if " "in person_name:
                                                        person_name=person_name.replace(' ',"")
                                                    if "\\" in person_name:
                                                        person_name=person_name.replace(' ',"")
                                                    if "." in person_name:
                                                        person_name=person_name.replace('.',"")
                                                    if "?" in person_name:
                                                        person_name=person_name.replace("?","")
                                                    if "!" in person_name:
                                                        person_name=person_name.replace("!","")
                                                    #remove accents, diacritics
                                                    person_name=unidecode.unidecode(person_name)
                                    ##########
                                    #Obtain the image
                                    #########
                                    if person_name!="":
                                        if "claims" in person["entities"][prefix].keys():
                                            if "P18" in person["entities"][prefix]["claims"].keys():
                                                if "mainsnak" in person["entities"][prefix]["claims"]["P18"][0].keys():
                                                    if "datavalue" in person["entities"][prefix]["claims"]["P18"][0]["mainsnak"].keys():
                                                        if "value" in person["entities"][prefix]["claims"]["P18"][0]["mainsnak"]["datavalue"].keys():
                                                            image_name=person["entities"][prefix]["claims"]["P18"][0]["mainsnak"]["datavalue"]["value"]
                                                            if '"' in image_name:
                                                                image_name=image_name.replace('"','')
                                                            if "\\" in image_name:
                                                                image_name=image_name.replace("\\","")
                                                            image_suffix="_".join(image_name.split(" "))
                                                            image_link="https://www.wikidata.org/wiki/%s#/media/File:%s"%(prefix,image_suffix)
                                                            #image_link = urllib.parse.quote(link,safe=':/')
                                                            try:
                                                                r=requests.get(image_link)
                                                            except requests.exceptions.Timeout:
                                                                print("Too much time to request image link %s"%(image_link))
                                                                flag=0
                                                            except requests.exceptions.TooManyRedirects:
                                                                print("Bad image link %s"%(image_link))
                                                                flag=0
                                                            except requests.exceptions.RequestException as e:
                                                                print("Catastrophic error for image link %s"%(image_link))
                                                                flag=0
                                                            if flag==1:
                                                                #parse HTML code
                                                                soup=BeautifulSoup(r.text,"html.parser")
                                                                #find all images in url (should only be 1 here)
                                                                images=soup.findAll('img')
                                                                if len(images)!=0:
                                                                    image=images[0]
                                                                    try:
                                                                        image_link=image["src"]
                                                                    except:
                                                                        flag=0
                                                                    if flag==1:
                                                                        image_link="https:"+image_link
                                                                        try:
                                                                            urllib.request.urlretrieve(image_link,save_dir+"\\"+person_name+".jpg")
                                                                        except Exception as e:
                                                                            print("%s for image has error %s, link:%s"%(image_name,e,image_link))
                                                                            flag=0
                                                                        if flag==1:
                                                                            #print("Success for %s with link %s"%(image_name,image_link))
                                                                            count_people+=1
                                                        
    count_links+=1
    if count_links%printing_iter==0:
        try:
            print("Downloaded %d images \t Tried %d links \t Last person: %s \t %s time"%(count_people,count_links,person_name,timming(start)))
        except NameError:
            pass
     


Downloaded 3 images 	 Tried 581900 links 	 Last person: MarioNasalliRoccadiCorneliano 	 0m 42s time
Downloaded 15 images 	 Tried 582000 links 	 Last person: ReneRedzepi 	 1m 37s time
Downloaded 22 images 	 Tried 582100 links 	 Last person: TakeshiSawada 	 2m 27s time
Downloaded 29 images 	 Tried 582200 links 	 Last person: HeliondeVilleneuve 	 3m 14s time
Downloaded 36 images 	 Tried 582300 links 	 Last person: WilliamRufusShafter 	 4m 5s time
Downloaded 44 images 	 Tried 582400 links 	 Last person: JohnBernardFitzpatrick 	 4m 56s time
Downloaded 54 images 	 Tried 582500 links 	 Last person: BartWellens 	 5m 49s time
Downloaded 60 images 	 Tried 582600 links 	 Last person: AndrewCashner 	 6m 39s time
Downloaded 68 images 	 Tried 582700 links 	 Last person: DionBoucicault 	 7m 31s time
Downloaded 76 images 	 Tried 582800 links 	 Last person: ChristianFerras 	 8m 20s time
Catastrophic error for image link https://www.wikidata.org/wiki/Q582890#/media/File:Fielding_L._Wright.jpg
Downloaded

Downloaded 946 images 	 Tried 591000 links 	 Last person: AntonKaufmann 	 81m 29s time
Downloaded 958 images 	 Tried 591100 links 	 Last person: SalvadorCapitano 	 82m 22s time
Downloaded 971 images 	 Tried 591200 links 	 Last person: JanayaStephens 	 83m 17s time
Downloaded 984 images 	 Tried 591300 links 	 Last person: AntonKrinner 	 84m 12s time
Downloaded 993 images 	 Tried 591400 links 	 Last person: AntonLandes 	 85m 2s time
Downloaded 1008 images 	 Tried 591500 links 	 Last person: AntonLeodegarKeller 	 85m 57s time
Downloaded 1022 images 	 Tried 591600 links 	 Last person: AntonLudwigSturler 	 86m 52s time
Downloaded 1044 images 	 Tried 591700 links 	 Last person: LogonaEsau 	 87m 56s time
Downloaded 1056 images 	 Tried 591800 links 	 Last person: AngelRozas 	 88m 51s time
Downloaded 1067 images 	 Tried 591900 links 	 Last person: AntonMayrhauser 	 89m 43s time
Downloaded 1073 images 	 Tried 592000 links 	 Last person: AntonMusa 	 90m 31s time
Downloaded 1082 images 	 Tried 592

Downloaded 1983 images 	 Tried 600200 links 	 Last person: SeanFitzpatrick 	 162m 25s time
Downloaded 1995 images 	 Tried 600300 links 	 Last person: SeanEadie 	 163m 22s time
Downloaded 2007 images 	 Tried 600400 links 	 Last person: KojiMiyata 	 164m 17s time
Downloaded 2013 images 	 Tried 600500 links 	 Last person: TomohiroHatta 	 165m 5s time
Downloaded 2021 images 	 Tried 600600 links 	 Last person: ImreSzabics 	 165m 53s time
Downloaded 2028 images 	 Tried 600700 links 	 Last person: Jean-BertrandPontalis 	 166m 44s time
Downloaded 2035 images 	 Tried 600800 links 	 Last person: PedroTafur 	 167m 33s time
Downloaded 2045 images 	 Tried 600900 links 	 Last person: TiborBalog 	 168m 28s time
Downloaded 2059 images 	 Tried 601000 links 	 Last person: JohanTuri 	 169m 25s time
Downloaded 2072 images 	 Tried 601100 links 	 Last person: AntonioCagnoni 	 170m 20s time
Downloaded 2084 images 	 Tried 601200 links 	 Last person: AntonioCantafora 	 171m 12s time
Downloaded 2104 images 	 Tr

Downloaded 3082 images 	 Tried 609400 links 	 Last person: AntsNisu 	 245m 50s time
Downloaded 3093 images 	 Tried 609500 links 	 Last person: LydiaRotich 	 246m 42s time
Downloaded 3107 images 	 Tried 609600 links 	 Last person: AnttiAarnio 	 247m 36s time
Downloaded 3124 images 	 Tried 609700 links 	 Last person: AnttiKuisma 	 248m 33s time
Downloaded 3136 images 	 Tried 609800 links 	 Last person: AnttiRautiola 	 249m 30s time
Downloaded 3152 images 	 Tried 609900 links 	 Last person: AnttiViitikko 	 250m 30s time
Downloaded 3162 images 	 Tried 610000 links 	 Last person: DmitriCheryshev 	 251m 24s time
Downloaded 3176 images 	 Tried 610100 links 	 Last person: CharlesPDixon 	 252m 20s time
Downloaded 3186 images 	 Tried 610200 links 	 Last person: JanHerman 	 253m 12s time
Downloaded 3193 images 	 Tried 610300 links 	 Last person: MichelleJenner 	 254m 2s time
Downloaded 3200 images 	 Tried 610400 links 	 Last person: ZulmaFaiad 	 254m 52s time
Downloaded 3208 images 	 Tried 610500

Downloaded 3934 images 	 Tried 617700 links 	 Last person: DenisGolovanov 	 325m 32s time
Downloaded 3938 images 	 Tried 617800 links 	 Last person: DavidGaul 	 326m 23s time
Downloaded 3944 images 	 Tried 617900 links 	 Last person: SuzuhitoYasuda 	 327m 17s time
Downloaded 3955 images 	 Tried 618000 links 	 Last person: LaszloBerti 	 328m 12s time
Downloaded 3962 images 	 Tried 618100 links 	 Last person: NatThewphaingam 	 329m 6s time
Downloaded 3962 images 	 Tried 618200 links 	 Last person: TetsuroOda 	 329m 49s time
Downloaded 3967 images 	 Tried 618300 links 	 Last person: HuseyinVelioglu 	 330m 35s time
Downloaded 3975 images 	 Tried 618400 links 	 Last person: ZhuZiqing 	 331m 29s time
Downloaded 3979 images 	 Tried 618500 links 	 Last person: AntonioFrancoFlorensa 	 332m 16s time
Downloaded 3982 images 	 Tried 618600 links 	 Last person: Apicius 	 333m 5s time
Downloaded 3988 images 	 Tried 618700 links 	 Last person: Mohammedal-Ajami 	 333m 56s time
Downloaded 3992 images 	 

...

Downloaded 7679 images 	 Tried 662500 links 	 Last person: DulalDutta 	 717m 2s time
Downloaded 7687 images 	 Tried 662600 links 	 Last person: VincentvanderVoort 	 717m 49s time
Catastrophic error for wikipaedia link https://upload.wikimedia.org/wikipedia/commons/thumb/b/be/Viktor_Tausk_%281900%29.jpg/220px-Viktor_Tausk_%281900%29.jpg
Downloaded 7696 images 	 Tried 662700 links 	 Last person: NenadStojkovic 	 718m 58s time
Downloaded 7703 images 	 Tried 662800 links 	 Last person: JanBrokoff 	 719m 45s time
Downloaded 7711 images 	 Tried 662900 links 	 Last person: TiborViniczai 	 720m 35s time
Downloaded 7719 images 	 Tried 663000 links 	 Last person: Anna-LenaFritzon 	 721m 23s time
Downloaded 7729 images 	 Tried 663100 links 	 Last person: PeterChristianKnudsen 	 722m 13s time
Downloaded 7737 images 	 Tried 663200 links 	 Last person: ShmuelShilo 	 722m 59s time
Downloaded 7741 images 	 Tried 663300 links 	 Last person: TheoHeemskerk 	 723m 41s time
Downloaded 7749 images 	 Tried 6

Downloaded 8449 images 	 Tried 670800 links 	 Last person: GiovanniD'Ercole 	 786m 33s time
Downloaded 8462 images 	 Tried 670900 links 	 Last person: LasseStrand 	 787m 24s time
Downloaded 8473 images 	 Tried 671000 links 	 Last person: HowardShultzMiller 	 788m 15s time
Downloaded 8484 images 	 Tried 671100 links 	 Last person: ArielJacobs 	 789m 5s time
Downloaded 8489 images 	 Tried 671200 links 	 Last person: MarcHottiger 	 789m 50s time
Downloaded 8503 images 	 Tried 671300 links 	 Last person: Amphibalus 	 790m 43s time
Downloaded 8512 images 	 Tried 671400 links 	 Last person: DavidDNewsom 	 791m 30s time
Downloaded 8522 images 	 Tried 671500 links 	 Last person: Jean-LucFournier 	 792m 19s time
Downloaded 8527 images 	 Tried 671600 links 	 Last person: GeorgGsell 	 793m 4s time
Downloaded 8544 images 	 Tried 671700 links 	 Last person: Swasawke 	 794m 2s time
Downloaded 8552 images 	 Tried 671800 links 	 Last person: JeronimoLobo 	 794m 50s time
Catastrophic error for wikipaed

Downloaded 9327 images 	 Tried 679200 links 	 Last person: Mentuemhat 	 856m 45s time
Downloaded 9336 images 	 Tried 679300 links 	 Last person: PierreMolinier 	 857m 34s time
Downloaded 9344 images 	 Tried 679400 links 	 Last person: PauloMenezes 	 858m 22s time
Downloaded 9350 images 	 Tried 679500 links 	 Last person: JosefDachs 	 859m 7s time
Downloaded 9360 images 	 Tried 679600 links 	 Last person: MoishePostone 	 859m 53s time
Downloaded 9376 images 	 Tried 679700 links 	 Last person: SisinioGonzalezMartinez 	 860m 49s time
Downloaded 9390 images 	 Tried 679800 links 	 Last person: DavideVanDeSfroos 	 861m 43s time
Downloaded 9396 images 	 Tried 679900 links 	 Last person: MarioScheiber 	 862m 26s time
Downloaded 9403 images 	 Tried 680000 links 	 Last person: LadislauBonyhadi 	 863m 11s time
Downloaded 9413 images 	 Tried 680100 links 	 Last person: AlainMieville 	 863m 57s time
Downloaded 9425 images 	 Tried 680200 links 	 Last person: LawrenceWeiner 	 864m 47s time
Downloaded

Catastrophic error for wikipaedia link https://upload.wikimedia.org/wikipedia/commons/thumb/f/f9/20160418_AUTITA_3468.jpg/220px-20160418_AUTITA_3468.jpg
Downloaded 10172 images 	 Tried 687800 links 	 Last person: ArminHorovitz 	 926m 19s time
Downloaded 10180 images 	 Tried 687900 links 	 Last person: MarioSoto 	 927m 5s time
Downloaded 10188 images 	 Tried 688000 links 	 Last person: DaleHennesy 	 927m 51s time
Downloaded 10194 images 	 Tried 688100 links 	 Last person: ArminMuller 	 928m 35s time
Downloaded 10205 images 	 Tried 688200 links 	 Last person: ArminRuhl 	 929m 24s time
Downloaded 10213 images 	 Tried 688300 links 	 Last person: ArminStorzenhofecker 	 930m 10s time
Downloaded 10230 images 	 Tried 688400 links 	 Last person: MarkusEibegger 	 931m 5s time
Downloaded 10241 images 	 Tried 688500 links 	 Last person: MaxStandfuss 	 931m 51s time
Downloaded 10249 images 	 Tried 688600 links 	 Last person: ArminiusHasemann 	 932m 37s time
Downloaded 10259 images 	 Tried 688700 li

KeyboardInterrupt: 